# Histórico de Bitácoras Onomásticas del SEMEFO-DF
# Limpieza de datos

1. Acceso a la información
2. Valores faltantes y duplicados
3. Eliminación de columnas
4. Limpieza de transcritos
5. Limpieza de nombres
6. Limpieza de categóricos
7. Limpieza de fecha
8. Limpieza de edad
9. Limpieza de procedencia
10. Limpieza de diagnósticos
11. Ordenar y guardar datos

-------

El **Servicio Médico Forense del Distrito Federal (SEMEFO-DF)** ahora [INCIFO-CdMx](https://www.incifocdmx.gob.mx/) es una institución oficial encargada de realizar autopsias y estudios forenses para esclarecer las causas de muerte, principalmente en casos de muertes violentas, sospechosas o relacionadas con hechos delictivos. 
El **Histórico de Bitácoras Onomásticas (HBO)** son registros administrativos utilizados por el SEMEFO-DF entre **1968 y 1982** para documentar el ingreso de cadáveres o restos humanos a sus instalaciones. 
Estas bitácoras contienen información individualizada sobre cada ingreso, como: nombre, fecha de ingreso, edad, diagnóstico de causa de muerte, expediente del SEMEFO, institución de procedencia y acta. 
Entre 2020 y 2023, la CNB accedió a estos documentos, resguardados por el INCIFO-CdMx, para digitalizar, transcribir e interpretar las anotaciones manuscritas. La base de datos se hizo pública por medio de la solicitud de acceso a la información 332163723000249 como un archivo en formato Excel.
Estos registros incluyen datos clave para identificar patrones históricos de mortalidad, enfermedades comunes y factores de riesgo, lo que las convierte en una fuente valiosa para estudios demográficos, epidemiológicos y de salud pública.

Este notebook documenta el proceso de limpieza al cual fue sometido el HBO. 
Se validaron los campos existentes, con cuidado de mantener la información original, incluyendo datos faltantes. 
Se desglosó la fecha y edad, se dividió los ingresos de tipo “cadáver” en “cadáver conocido” y “cadáver desconocido”  con base en el campo `Conocido_desconocido` y se estandarizaron los campos de diagnóstico y procedencia utilizando los catálogos correspondientes.  
Se incluyen todos los registros que correspondieran a ingresos y se eliminaron las filas correspondientes a renglones vacíos en los documentos manuscritos. 
El resultado es un conjunto de datos con 96825 ingresos y 32 variables, el cual corresponde a la población de todos los ingresos de restos al SEMEFO-DF en el periodo 1968-1982.

### Acceso a la información

Se puede acceder al archivo original a través de la [Plataforma de Transparencia](https://www.plataformadetransparencia.org.mx/Inicio) solicitud 332163723000249 o a la copia resguardada en [datamx](https://datamx.io/dataset/ingresos-del-semefo-df-1965-1982-inai).

Al descargar obtendremos un archivo llamado `CNB_DOB_BPGS_Respuesta_Solicitud 332163723000249.xlsx`, el cual contiene las siguientes hojas:
* `Solicitud_folio_332163723000249`: información del conjunto de datos y aclaraciones útiles
* `Diccionario`: descripción del conjunto de datos
* `HBO`: tabla con la información del Histórico de las Bitácoras Onomásticas (HBO)

In [1]:
from numpy import nan
import pandas as pd
from unidecode import unidecode

filename = 'data_raw/CNB_DOB_BPGS_Respuesta_Solicitud 332163723000249.xlsx'
df = pd.read_excel(filename, sheet_name="HBO", index_col='ID', dtype='str')
df

,Numero_progresivo_transcrito,Nombre_completo_transcrito,Primer_apellido,Segundo_apellido,Nombres_propios,Fecha_transcrito,Fecha_estandar,Expediente_SEMEFO_transcrito,Procedencia_transcrito,Procedencia_estandar,...,Diagnostico_estandar,Diagnostico_extendido,Sexo,Edad_transcrito,Tipo_restos,Bitacora_ingresos,Pagina_PDF,Foja_transcrito,Observaciones,Conocido_desconocido
ID,,,,,,,,,,,,,,,,,,,,,
BO_1968_00001,S-D,acosta ortega teresa,acosta,ortega,teresa,1968-01-03,1968-01-03 00:00:00,37,S-D,S-D,...,S-D,sin datos,Femenino,S-D,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00002,S-D,avila de cuestas catalina,avila,de cuestas,catalina,1968-01-05,1968-01-05 00:00:00,58,S-D,S-D,...,S-D,sin datos,Femenino,S-D,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00003,S-D,arzate paredes juan,arzate,paredes,juan,1968-01-07,1968-01-07 00:00:00,83,S-D,S-D,...,S-D,sin datos,Masculino,S-D,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00004,S-D,alvarez martinez isaac,alvarez,martinez,isaac,1968-01-07,1968-01-07 00:00:00,86,S-D,S-D,...,S-D,sin datos,Masculino,S-D,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00005,S-D,arellano viuda de campos ma.,arellano,viuda de campos,ma.,1968-01-07,1968-01-07 00:00:00,88,S-D,S-D,...,S-D,sin datos,Femenino,S-D,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BO_1982_07489,S-D,placenta,s-d,s-d,s-d,1982-06-05,1982-06-05 00:00:00,3079,15a,15A,...,S-D,sin datos,S-D,S-D,Miembros,semefo_df_bo_1982,251,156,NaN,desconocido
BO_1982_07490,S-D,5 dedos del pie derecho de desconocido,s-d,s-d,s-d,1982-06-05,1982-06-05 00:00:00,3060,32a,32A,...,S-D,sin datos,S-D,S-D,Miembros,semefo_df_bo_1982,251,156,NaN,desconocido
BO_1982_07491,S-D,dedo de desconocido,s-d,s-d,s-d,1982-11-19,1982-11-19 00:00:00,6389,32a,32A,...,S-D,sin datos,S-D,S-D,Miembros,semefo_df_bo_1982,251,156,NaN,desconocido


Generemos un profile de datos inicial

## Valores faltantes y duplicados

Datos faltantes por columna o variable

In [2]:
df.isna().sum()

Numero_progresivo_transcrito        0
Nombre_completo_transcrito          0
Primer_apellido                  1878
Segundo_apellido                  281
Nombres_propios                  2095
Fecha_transcrito                    0
Fecha_estandar                    266
Expediente_SEMEFO_transcrito        0
Procedencia_transcrito              0
Procedencia_estandar              408
Procedencia_direccion             437
Procedencia_alcaldia            31946
Numero_acta_transcrito              0
Procedencia_acta                25635
Diagnostico_transcrito              0
Diagnostico_estandar             8267
Diagnostico_extendido            8267
Sexo                                0
Edad_transcrito                     0
Tipo_restos                         0
Bitacora_ingresos                   0
Pagina_PDF                          0
Foja_transcrito                     0
Observaciones                   82723
Conocido_desconocido                0
dtype: int64

Datos faltantes por columna u observación

Hay 406 filas donde faltan 5 o más valores. La mayor parte de los datos faltantes corresponden a casos donde se borraron los nombres para proteger datos personales, aunque también hay datos faltantes en las observaciones y el diagnóstico, estos `nan`s representan nombres borrados por la CNB por protección de datos.

In [3]:
df.isna().sum(axis=1).value_counts().sort_index()

0     9821
1    46700
2     7775
3    30780
4     1362
5      281
6      117
7        7
9        1
Name: count, dtype: int64

En el caso de los nombres podemos ver varios casos de datos faltantes.  

* "s-d": de acuerdo a la información proporcionada con el conjunto de datos, esto significa que el espacio que corresponde al nombre se encuentra vacío en las Bitácoras originales, y por lo tanto se transcribió como s-d para reflejar la falta de información de origen
* "desconocido" y "desconocida": en este caso, el SEMEFO desconoce la identidad del ingreso
* "Nombre de particular que podría encontrarse con vida. Se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.": en este caso el nombre se oculta por protección de datos personales
* `nan` esto sucede cuando la celda está vacía, en este caso no hay celdas vacías

In [4]:
df['Nombre_completo_transcrito'].value_counts().head(10).to_frame()

,count
Nombre_completo_transcrito,
desconocido,12828
feto,2610
Nombre de particular que podría encontrarse con vida. Se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.,2158
desconocida,1294
r. nacido,224
recien nacido,105
r. nacida,96
recien nacida,61
r nacido,34


Registros duplicados. En total hay 227 filas con registros duplicados correspondientes a 114 registros únicos.

Estos tienen identificador único, por lo que se mantendrán cómo están.

In [5]:
df[ df.duplicated(keep=False) ]

,Numero_progresivo_transcrito,Nombre_completo_transcrito,Primer_apellido,Segundo_apellido,Nombres_propios,Fecha_transcrito,Fecha_estandar,Expediente_SEMEFO_transcrito,Procedencia_transcrito,Procedencia_estandar,...,Diagnostico_estandar,Diagnostico_extendido,Sexo,Edad_transcrito,Tipo_restos,Bitacora_ingresos,Pagina_PDF,Foja_transcrito,Observaciones,Conocido_desconocido
ID,,,,,,,,,,,,,,,,,,,,,
BO_1968_01734,S-D,gomez caballero pedro,gomez,caballero,pedro,1968-07-11,1968-07-11 00:00:00,2480,S-D,S-D,...,S-D,sin datos,Masculino,S-D,Cadáver,semefo_df_bo_1968,60,39,NaN,conocido
BO_1968_01736,S-D,gomez caballero pedro,gomez,caballero,pedro,1968-07-11,1968-07-11 00:00:00,2480,S-D,S-D,...,S-D,sin datos,Masculino,S-D,Cadáver,semefo_df_bo_1968,60,39,NaN,conocido
BO_1968_02175,S-D,desconocido,s-d,s-d,s-d,1968-07-11,1968-07-11 00:00:00,2479,11a Del.,11A,...,S-D,sin datos,Masculino,S-D,Cadáver,semefo_df_bo_1968,74,49,NaN,desconocido
BO_1968_02177,S-D,desconocido,s-d,s-d,s-d,1968-07-11,1968-07-11 00:00:00,2479,11a Del.,11A,...,S-D,sin datos,Masculino,S-D,Cadáver,semefo_df_bo_1968,74,49,NaN,desconocido
BO_1968_02595,S-D,desconocido,s-d,s-d,s-d,1968-12-03,1968-12-03 00:00:00,4393,1a Del.,1A,...,S-D,sin datos,Masculino,S-D,Cadáver,semefo_df_bo_1968,88,56 reverso,NaN,desconocido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BO_1982_05377,S-D,sanchez carranza isaias,sanchez,carranza,isaias,1982-10-12,1982-10-12 00:00:00,5657,HB,HTB,...,S-D,sin datos,Masculino,S-D,Cadáver,semefo_df_bo_1982,180,108 reverso,NaN,conocido
BO_1982_05378,S-D,suarez hernandez jesus,suarez,hernandez,jesus,1982-10-11,1982-10-11 00:00:00,5644,15a,15A,...,S-D,sin datos,Masculino,S-D,Cadáver,semefo_df_bo_1982,180,108 reverso,NaN,conocido
BO_1982_05379,S-D,salazar patino javier,salazar,patino,javier,1982-10-09,1982-10-09 00:00:00,5589,1a,1A,...,S-D,sin datos,Masculino,S-D,Cadáver,semefo_df_bo_1982,180,108 reverso,NaN,conocido


## Eliminación de columnas

Se quitaran las columnas de fecha, procedencia y diagnóstico para regenerar.

In [6]:
col_drop = [ 'Fecha_estandar', 'Procedencia_estandar', 'Procedencia_direccion', 'Procedencia_alcaldia','Procedencia_acta', 
             'Diagnostico_estandar', 'Diagnostico_extendido', 'Pagina_PDF'  ]
df = df.drop( col_drop, axis=1)
df.head()

,Numero_progresivo_transcrito,Nombre_completo_transcrito,Primer_apellido,Segundo_apellido,Nombres_propios,Fecha_transcrito,Expediente_SEMEFO_transcrito,Procedencia_transcrito,Numero_acta_transcrito,Diagnostico_transcrito,Sexo,Edad_transcrito,Tipo_restos,Bitacora_ingresos,Foja_transcrito,Observaciones,Conocido_desconocido
ID,,,,,,,,,,,,,,,,,
BO_1968_00001,S-D,acosta ortega teresa,acosta,ortega,teresa,1968-01-03,37,S-D,S-D,S-D,Femenino,S-D,Cadáver,semefo_df_bo_1968,1,NaN,conocido
BO_1968_00002,S-D,avila de cuestas catalina,avila,de cuestas,catalina,1968-01-05,58,S-D,S-D,S-D,Femenino,S-D,Cadáver,semefo_df_bo_1968,1,NaN,conocido
BO_1968_00003,S-D,arzate paredes juan,arzate,paredes,juan,1968-01-07,83,S-D,S-D,S-D,Masculino,S-D,Cadáver,semefo_df_bo_1968,1,NaN,conocido
BO_1968_00004,S-D,alvarez martinez isaac,alvarez,martinez,isaac,1968-01-07,86,S-D,S-D,S-D,Masculino,S-D,Cadáver,semefo_df_bo_1968,1,NaN,conocido
BO_1968_00005,S-D,arellano viuda de campos ma.,arellano,viuda de campos,ma.,1968-01-07,88,S-D,S-D,S-D,Femenino,S-D,Cadáver,semefo_df_bo_1968,1,NaN,conocido


## Limpieza de transcritos

Las columnas transcritas de origen no se van a modificar, pero es importante verificar el formato.

Nota: desde Excel se forzó el formato de la columna 'Fecha_transcrito' a ser texto.

In [7]:
cols_trans = ['Numero_progresivo_transcrito', 'Nombre_completo_transcrito', 'Fecha_transcrito', 'Expediente_SEMEFO_transcrito', 
              'Procedencia_transcrito', 'Numero_acta_transcrito', 'Diagnostico_transcrito', 'Edad_transcrito', 'Foja_transcrito', 'Observaciones']
for col in cols_trans:
    df[col] = df[col].apply(str).replace({'nan':nan, 's-d':'S-D'})

En el caso de 'Nombre_completo_transcrito' sustituiremos la leyenda 'Nombre de particular que podría encontrarse con vida. Se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP' por 'nombre confidencial' y 's-d' por 'S-D'.

In [8]:
replace_str = {'Nombre de particular que podría encontrarse con vida. Se clasifica como confidencial con fundamento en el artículo 116 de la LGTAIP.':'nombre confidencial' }
# Aplicamos función a columna de interes
df['Nombre_completo_transcrito'] = df['Nombre_completo_transcrito'].replace(replace_str)
# Revisamos resultado
df['Nombre_completo_transcrito'].value_counts(dropna=False).head()

Nombre_completo_transcrito
desconocido            12828
feto                    2610
nombre confidencial     2158
desconocida             1294
r. nacido                224
Name: count, dtype: int64

## Limpieza de nombres

En el caso de 'Primer_apellido', 'Segundo_apellido' y 'Nombres_propios' sustituiremos los `nan` por 'NC' (nombre confidencial) y los 's-d' por `np.nan`. En este caso el orden de las operaciones es importante.

In [9]:
col_str = ['Primer_apellido', 'Segundo_apellido', 'Nombres_propios']

for col in col_str:
    df[col] = df[col].fillna('NC')
    df[col] = df[col].replace({'s-d':nan})
df['Primer_apellido'].value_counts(dropna=False)

Primer_apellido
NaN          18107
hernandez     2919
garcia        2536
martinez      2328
gonzalez      1904
             ...  
orejas           1
nateras          1
nopal            1
neguez           1
palo             1
Name: count, Length: 7798, dtype: int64

Para facilitar el análisis, se quitaran acentos, espacios extra y verificara que esten en minusculas los nombres.

In [10]:
def limpiar_texto(s:str, capitalize:str=None, dic_replace:dict=None):
    if type(s)==str: # checar el tipo
        s = ' '.join( s.split() ) #quitar espacios extra
        s = unidecode(s) #quitar acentos
        if capitalize!=None: # cambiar capitalizacion
            if capitalize=='upper':  s = s.upper()
            elif capitalize=='lower':  s = s.lower()
            elif capitalize=='title':  s = s.title()
            elif capitalize=='capitalize':  s = s.capitalize()
    return s

for col in col_str:
    df[col] = df[col].apply( limpiar_texto, capitalize='lower' )
    
df[col_str]

,Primer_apellido,Segundo_apellido,Nombres_propios
ID,,,
BO_1968_00001,acosta,ortega,teresa
BO_1968_00002,avila,de cuestas,catalina
BO_1968_00003,arzate,paredes,juan
BO_1968_00004,alvarez,martinez,isaac
BO_1968_00005,arellano,viuda de campos,ma.
...,...,...,...
BO_1982_07489,NaN,NaN,NaN
BO_1982_07490,NaN,NaN,NaN
BO_1982_07491,NaN,NaN,NaN


## Limpieza de categóricos

En este conjunto las columnas categóricas son: 'Sexo', 'Tipo_restos', 'Conocido_desconocido', 'Bitacora_ingresos'

In [11]:
col_cat = ['Bitacora_ingresos', 'Sexo', 'Tipo_restos', 'Conocido_desconocido']

for col in col_cat:
    print(col)
    display( df[col].unique() )

Bitacora_ingresos


array(['semefo_df_bo_1968', 'semefo_df_bo_1969', 'semefo_df_bo_1970',
       'semefo_df_bo_1971', 'semefo_df_bo_1972', 'semefo_df_bo_1973',
       'semefo_df_bo_1974', 'semefo_df_bo_1975', 'semefo_df_bo_1976',
       'semefo_df_bo_1977', 'semefo_df_bo_1978', 'semefo_df_bo_1979',
       'semefo_df_bo_1980', 'semefo_df_bo_1981', 'semefo_df_bo_1982'],
      dtype=object)

Sexo


array(['Femenino', 'Masculino', 'S-D'], dtype=object)

Tipo_restos


array(['Cadáver', 'Miembros', 'Feto', 'Restos óseos', 'Recién nacido'],
      dtype=object)

Conocido_desconocido


array(['conocido', 'desconocido', 'S-D'], dtype=object)

Estandarizar datos de Conocido_desconocido y renombrar a Identificacion

In [12]:
df = df.rename(columns={'Conocido_desconocido':'Identificacion'})
df['Identificacion'] = df['Identificacion'] \
                                .replace( {'conocido':'Conocido', 'desconocido':'Desconocido'} ) \
                                .astype("category")
df['Identificacion'].head()

ID
BO_1968_00001    Conocido
BO_1968_00002    Conocido
BO_1968_00003    Conocido
BO_1968_00004    Conocido
BO_1968_00005    Conocido
Name: Identificacion, dtype: category
Categories (3, object): ['Conocido', 'Desconocido', 'S-D']

En este caso agruparemos los restos óseos con los miembros.

In [13]:
from pandas.api.types import CategoricalDtype

df.loc[(df['Tipo_restos']=='Cadáver') & (df['Identificacion']=='Conocido'), 'Tipo_restos'] = 'Cadáver conocido'
df.loc[(df['Tipo_restos']=='Cadáver') & (df['Identificacion']=='Desconocido'), 'Tipo_restos']  = 'Cadáver desconocido'
df['Tipo_restos'].value_counts(normalize=True, dropna=False)

orden_restos = ['Cadáver conocido', 'Cadáver desconocido', 'Recién nacido', 'Feto', 'Miembros', 'Restos óseos']
orden_restos = CategoricalDtype(categories=orden_restos, ordered=True)
df['Tipo_restos'] = df['Tipo_restos'].astype( orden_restos )
df['Tipo_restos'].tail()

ID
BO_1982_07489               Miembros
BO_1982_07490               Miembros
BO_1982_07491               Miembros
BO_1982_07492               Miembros
BO_1982_07493    Cadáver desconocido
Name: Tipo_restos, dtype: category
Categories (6, object): ['Cadáver conocido' < 'Cadáver desconocido' < 'Recién nacido' < 'Feto' < 'Miembros' < 'Restos óseos']

In [14]:
for col in ['Bitacora_ingresos', 'Sexo']:
    df[col] = df[col].astype('category')
df.dtypes

Numero_progresivo_transcrito      object
Nombre_completo_transcrito        object
Primer_apellido                   object
Segundo_apellido                  object
Nombres_propios                   object
Fecha_transcrito                  object
Expediente_SEMEFO_transcrito      object
Procedencia_transcrito            object
Numero_acta_transcrito            object
Diagnostico_transcrito            object
Sexo                            category
Edad_transcrito                   object
Tipo_restos                     category
Bitacora_ingresos               category
Foja_transcrito                   object
Observaciones                     object
Identificacion                  category
dtype: object

## Limpieza de fecha

Las fechas se encuentran en formato `datetime64`, lo cual incluye, fecha y hora.
Este formato sigue el patrón: `yyyy-mm-dd hh:mm:ss`

En este conjunto las columnas de tiempo son:
* 'Fecha_transcrito'

In [15]:
df['Fecha_estandar'] = pd.to_datetime(df['Fecha_transcrito'], errors='coerce')
display(df.loc[ (df['Fecha_transcrito'].notna()) & (df['Fecha_estandar'].isna()), ['Fecha_transcrito', 'Fecha_estandar'] ].drop_duplicates())

,Fecha_transcrito,Fecha_estandar
ID,,
BO_1968_02771,S-D,NaT
BO_1970_01167,1970-04-31,NaT
BO_1970_02996,1970-02-29,NaT
BO_1971_00319,1971-02-29,NaT
BO_1972_05531,1972-02-30,NaT
BO_1974_02008,1974-02-29,NaT
BO_1976_00456,1976-04-31,NaT
BO_1976_00588,1976-11-31,NaT
BO_1976_01290,1976-06-31,NaT


En este caso se puede ver que la conversión fallo solo en dos tipos de casos:
* Información incompleta '1979-01'
* Fecha no existente: '1979-02-31'

En este caso, ya que no hay forma de corregir estos errores, dejaremos las fechas cómo `nan`

Agregaremos columnas para marcar el día de la semana, del año y número de semanas.

In [16]:
df['Fecha_año'] = df['Fecha_estandar'].dt.year
df['Fecha_mes'] = df['Fecha_estandar'].dt.month
df['Fecha_semana'] = df['Fecha_estandar'].dt.isocalendar().week
df['Fecha_diasemana'] = df['Fecha_estandar'].dt.weekday

dic_semana = {0:'Lunes', 1:'Martes', 2:'Miercoles', 3:'Jueves',
              4:'Viernes', 5:'Sábado', 6:'Domingo'}
df['Fecha_diasemana'] = df['Fecha_diasemana'].replace(dic_semana)
df['Fecha_diasemana'].unique()

orden_semana = dic_semana.values()
orden_semana = CategoricalDtype(categories=orden_semana, ordered=True)

df['Fecha_diasemana'] = df['Fecha_diasemana'].astype( orden_semana )
df['Fecha_semana'] = df['Fecha_semana'].astype("category")
df['Fecha_mes'] = df['Fecha_mes'].astype("category")
df['Fecha_año'] = df['Fecha_año'].astype("category")
df.dtypes

Numero_progresivo_transcrito            object
Nombre_completo_transcrito              object
Primer_apellido                         object
Segundo_apellido                        object
Nombres_propios                         object
Fecha_transcrito                        object
Expediente_SEMEFO_transcrito            object
Procedencia_transcrito                  object
Numero_acta_transcrito                  object
Diagnostico_transcrito                  object
Sexo                                  category
Edad_transcrito                         object
Tipo_restos                           category
Bitacora_ingresos                     category
Foja_transcrito                         object
Observaciones                           object
Identificacion                        category
Fecha_estandar                  datetime64[ns]
Fecha_año                             category
Fecha_mes                             category
Fecha_semana                          category
Fecha_diasema

## Limpieza de edad

En este conjunto de datos, la columna `Edad_transcrito` contiene números; sin embargo, su tipo es `object`, ya que hay edades como '6 días' o algunas que incluyen posibles errores cómo '-48' o ' S-D'.

In [17]:
# columna de trabajo str
df['Edad_str'] = df['Edad_transcrito'].apply( limpiar_texto, capitalize='lower' )
df['Edad_str'] = df['Edad_str'].replace('s-d',nan)
# columna de trabajo int
df['Edad_int'] = pd.to_numeric(df['Edad_str'], errors="coerce")
# revisión errores de conversión
df.loc[ (df['Edad_str'].notna()) & (df['Edad_int'].isna()), ['Tipo_restos','Edad_str','Edad_int'] ].drop_duplicates()

,Tipo_restos,Edad_str,Edad_int
ID,,,
BO_1971_01274,Cadáver conocido,6 dias,NaN
BO_1973_00107,Recién nacido,16 dias,NaN
BO_1973_01155,Cadáver desconocido,30 dias,NaN
BO_1973_02920,Cadáver desconocido,4 meses,NaN
BO_1973_03132,Cadáver desconocido,3 meses,NaN
...,...,...,...
BO_1979_07182,Feto,35 semanas,NaN
BO_1979_07221,Recién nacido,5 dias,NaN
BO_1979_07222,Recién nacido,10 dias,NaN


La mayor parte de las edades en semanas son de fetos y recién nacido. En el caso de los fetos y recién nacidos podemos suponer que la edad es cero. Sin embargo, solo debemos de hacer esa suposición en el caso de los que tienen una edad registrada, para evitar cambiar la distribución de la población. 

Las celdas a convertir son aquellas que cumplen las condiciones:
* Existe un texto en 'Edad_str'
* El tipo es Feto o Recién nacido

In [18]:
row_fetorn = (df['Edad_str'].notna()) & (df['Tipo_restos'].isin(['Feto','Recién nacido']))
df.loc[row_fetorn, 'Edad_int'] = 0
df.loc[row_fetorn,['Edad_str', 'Tipo_restos', 'Edad_int']]

,Edad_str,Tipo_restos,Edad_int
ID,,,
BO_1973_00107,16 dias,Recién nacido,0.0
BO_1974_01307,38,Feto,0.0
BO_1974_05298,9 meses,Feto,0.0
BO_1974_05299,6 meses,Feto,0.0
BO_1974_05300,7 meses,Feto,0.0
...,...,...,...
BO_1979_07222,10 dias,Recién nacido,0.0
BO_1979_07223,-72,Recién nacido,0.0
BO_1979_07224,-72,Recién nacido,0.0


En este caso haremos una sustitución semiautomática, la cual hemos guardado en un archivo por el tamaño.
En este caso es muy importante el orden de las operaciones. Por ejemplo, hay '1' tanto en fetos cómo en cadáveres. En este caso podemos suponer por el contexto que representan cuestiones diferentes.

In [19]:
# Correción por catálogo
dic_edad = pd.read_csv('catalogos/cat_edad_sustitucion.csv')
display( dic_edad.head() )
dic_edad = {k:v for k,v in zip(dic_edad['edad_str'],dic_edad['edad_int'])}
df['Edad_estandar'] = df['Edad_str'].replace(dic_edad)
# Edad especial a fetos y RN
df.loc[(df['Edad_estandar'].notna()) & (df['Tipo_restos'].isin(['Feto','Recién nacido'])) , 'Edad_estandar'] = 0
# Convertir a entero
df['Edad_estandar'] = pd.to_numeric(df['Edad_estandar']).round()
df['Edad_estandar'].unique()

,edad_str,edad_int
0,6 dias,0.0
1,30 dias,0.0
2,4 meses,0.0
3,3 meses,0.0
4,14 dias,0.0


array([ nan,   0.,  63.,  17.,  69.,  40.,  68.,  50.,  42.,  66.,  55.,
        60.,  30.,  48.,  39.,  35.,  37.,  56.,  62.,  27.,   4.,  41.,
        25.,  72.,  16.,  65.,  26.,   8.,  20.,   3.,  28.,  58.,  23.,
        22.,  75.,   1.,  24.,  54.,  19.,  34.,  80.,  13.,  36.,  45.,
         2.,  61.,  12.,  70.,  49.,  95.,  33.,  32.,  14.,  57.,  85.,
        21.,   5.,  74.,   7.,  15.,   6.,  86.,  11.,  47.,  29.,  52.,
        81.,  84.,  18.,  38.,  43.,  53.,   9.,  44.,  64.,  73.,  82.,
        83.,  51.,  77.,  10.,  31.,  46.,  59.,  79.,  71.,  67.,  78.,
        90.,  76.,  98.,  93.,  87.,  88.,  89.,  94.,  92., 103.,  91.,
        96.,  97., 102., 100.,  99., 108., 116., 107.])

Veamos la distribución de edades:

In [20]:
df['Edad_estandar'].describe()

count    40172.000000
mean        35.236782
std         22.130117
min          0.000000
25%         20.000000
50%         34.000000
75%         50.000000
max        116.000000
Name: Edad_estandar, dtype: float64

Tenemos una persona de 116 años, esto podría ser un dato fuera de rango. 
En este caso vamos a poner un criterio para manejar problemas similares a futuro, si la persona tiene más de ciento diez años remplazaremos su edad por `nan`.

In [21]:
df.loc[ df['Edad_estandar']>=110,'Edad_estandar' ] = nan
df['Edad_estandar'].describe()

count    40171.000000
mean        35.234771
std         22.126724
min          0.000000
25%         20.000000
50%         34.000000
75%         50.000000
max        108.000000
Name: Edad_estandar, dtype: float64

Dividiremos las edades por grupo de edad, definiendo los bins de manera explícita para que coincidan con los del INEGI. Nota cómo iniciamos la serie en -1 y la terminamos en 110 para manejar los valores extremos del rango. Además, se tuvo cuidado de incluir un bin especial para los menores de un año `[0, 1)`. 

In [22]:
bins = [0, 1] + list(range(5, 86, 5)) + [110]
df['Edad_grupo'] = pd.cut(df['Edad_estandar'], bins, right=False)
df['Edad_grupo'].dropna()

ID
BO_1971_01274      [0, 1)
BO_1973_00107      [0, 1)
BO_1973_01155      [0, 1)
BO_1973_02920      [0, 1)
BO_1973_03132      [0, 1)
                   ...   
BO_1979_07440    [20, 25)
BO_1979_07442    [15, 20)
BO_1979_07451    [20, 25)
BO_1979_07452    [65, 70)
BO_1979_07457    [20, 25)
Name: Edad_grupo, Length: 40171, dtype: category
Categories (19, interval[int64, left]): [[0, 1) < [1, 5) < [5, 10) < [10, 15) ... [70, 75) < [75, 80) < [80, 85) < [85, 110)]

## Limpieza de procedencia
En algunos casos es necesario generar una tabla con equivalencias e información extra.

Por ejemplo, hay múltiples formas de escribir una misma procedencia. La Coordinación 1 de Cuauhtémoc aparece cómo '1a', 'CH-1', etc. Además, hay casos donde los ingresos provienen de múltiples procedencias, cómo '33a+1a' o 'C-I HX'. En este caso es necesario hacer procesos de limpieza complejos que requieren revisión manual. El resultado de ese proceso es la tabla `cat_proc_sustitucion.xlsx`.

In [23]:
df_proc = pd.read_excel('catalogos/cat_proc_sustitucion.xlsx')
df_proc.tail()

,Procedencia_transcrito,Procedencia_estandar,freq,Npartes,Nfalta,IDs,siglas1,siglas2,siglas3,extendido1,...,extendido3,Procedencia_extendido,alcaldia1,alcaldia2,alcaldia3,Procedencia_alcaldia,clasificacion1,clasificacion2,clasificacion3,Procedencia_clasificacion
854,V. Obregón,VO,1,1,0,BO_1972_01905,VO,NaN,NaN,Villa Obregon,...,NaN,Villa Obregon,Alvaro Obregon,NaN,NaN,Alvaro Obregon,NaN,NaN,NaN,NaN
855,VO-1,VO,1,1,0,BO_1975_00590,VO,NaN,NaN,Villa Obregon,...,NaN,Villa Obregon,Alvaro Obregon,NaN,NaN,Alvaro Obregon,NaN,NaN,NaN,NaN
856,V.o.,VO,1,1,0,BO_1973_03000,VO,NaN,NaN,Villa Obregon,...,NaN,Villa Obregon,Alvaro Obregon,NaN,NaN,Alvaro Obregon,NaN,NaN,NaN,NaN
857,Vo,VO,1,1,0,BO_1973_01821,VO,NaN,NaN,Villa Obregon,...,NaN,Villa Obregon,Alvaro Obregon,NaN,NaN,Alvaro Obregon,NaN,NaN,NaN,NaN
858,H.V.O.,VO,1,1,0,BO_1968_00866,VO,NaN,NaN,Villa Obregon,...,NaN,Villa Obregon,Alvaro Obregon,NaN,NaN,Alvaro Obregon,NaN,NaN,NaN,NaN


Uniendo con el HBO

In [24]:
df = df.reset_index() # conserva indice
cols = ['Procedencia_transcrito','Procedencia_estandar','Procedencia_extendido','Procedencia_alcaldia','Procedencia_clasificacion']
df = df.merge(df_proc[cols], how='left', on='Procedencia_transcrito')
df[['ID'] +cols]

,ID,Procedencia_transcrito,Procedencia_estandar,Procedencia_extendido,Procedencia_alcaldia,Procedencia_clasificacion
0,BO_1968_00001,S-D,S-D,Sin datos,NaN,NaN
1,BO_1968_00002,S-D,S-D,Sin datos,NaN,NaN
2,BO_1968_00003,S-D,S-D,Sin datos,NaN,NaN
3,BO_1968_00004,S-D,S-D,Sin datos,NaN,NaN
4,BO_1968_00005,S-D,S-D,Sin datos,NaN,NaN
...,...,...,...,...,...,...
96839,BO_1982_07489,15a,GAM-2,Coord Territorial 2 Gustavo A Madero (Col Arag...,Gustavo A Madero,Judicial
96840,BO_1982_07490,32a,COY-2,Coord Territorial 2 Coyoacan (Col Romero de Te...,Coyoacan,Judicial
96841,BO_1982_07491,32a,COY-2,Coord Territorial 2 Coyoacan (Col Romero de Te...,Coyoacan,Judicial
96842,BO_1982_07492,27a,XO-2,Coord Territorial 2 Xochimilco (Barrio de San ...,Xochimilco,Judicial


A continuación haremos varios pasos extra de limpieza. Cómo estás columnas son principalmente para el análisis sustituiremos los 'S-D' por `nan`. Además, volveremos las columnas categóricas. 

In [25]:
cols = ['Procedencia_estandar','Procedencia_extendido']
df[cols] = df[cols].replace({'S-D':nan, 'Sin datos':nan})

for col in ['Procedencia_estandar','Procedencia_alcaldia','Procedencia_clasificacion']:
    df[col] = df[col].astype("category")

df[cols].value_counts(dropna=False).head(10)

Procedencia_estandar  Procedencia_extendido                                     
NaN                   NaN                                                           25780
HB                    Hospital Balbuena (Col Jardin Balbuena)                        5440
CR                    Cruz Roja (Col Polanco)                                        4874
GAM-2                 Coord Territorial 2 Gustavo A Madero (Col Aragon La Villa)     4107
COY-2                 Coord Territorial 2 Coyoacan (Col Romero de Terreros)          3635
CUH-1                 Coord Territorial 1 Cuauhtemoc (Col San Simon Tolnahuac)       3583
HX                    Hospital Xoco (Col General Anaya)                              3573
HV                    Hospital la Villa (Col Granjas Modernas)                       3505
HGRL                  Hospital Ruben Leñero (Col Santo Tomas)                        2974
CUH-6                 Coord Territorial 6 Cuauhtemoc (Col Centro)                    2311
Name: count, dtype:

## Limpieza de diagnósticos

Repetiremos el mismo procedimiento para los diagnósticos.

In [26]:
df_diag = pd.read_excel('catalogos/cat_diag_sustitucion.xlsx')
df_diag.tail()

,Diagnostico_transcrito,Diagnostico_estandar,freq,Npartes,Nfalta,IDs,siglas1,siglas2,siglas3,siglas4,...,extendido3,extendido4,extendido5,Diagnostico_extendido,clasificacion1,clasificacion2,clasificacion3,clasificacion4,clasificacion5,Diagnostico_clasificacion
3704,HPAF C/S Vasos,AN+HPAF,1,2,0,BO_1975_01878,AN,HPAF,NaN,NaN,...,NaN,NaN,NaN,anemia + herida arma fuego,"Sintomas, signos y hallazgos anormales",Herida arma de fuego,NaN,NaN,NaN,"Herida arma de fuego + Sintomas, signos y hall..."
3705,HPAF C-Sec ArtF,AN+HPAF,1,2,0,BO_1975_02943,AN,HPAF,NaN,NaN,...,NaN,NaN,NaN,anemia + herida arma fuego,"Sintomas, signos y hallazgos anormales",Herida arma de fuego,NaN,NaN,NaN,"Herida arma de fuego + Sintomas, signos y hall..."
3706,HPAF AQxT,AQXT,1,1,0,BO_1979_07232,AQXT,NaN,NaN,NaN,...,NaN,NaN,NaN,amputacion quirurgica por traumatismo,Amputación,NaN,NaN,NaN,NaN,Amputación
3707,HPAF (Cuello),HPAFCu,1,1,0,BO_1974_00667,HPAFCu,NaN,NaN,NaN,...,NaN,NaN,NaN,herida arma fuego cuello,Herida arma de fuego,NaN,NaN,NaN,NaN,Herida arma de fuego
3708,HIsCPT,HICPT,1,1,1,BO_1977_03542,HICPT,NaN,NaN,NaN,...,NaN,NaN,NaN,HICPT,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
cols = ['Diagnostico_transcrito','Diagnostico_estandar','Diagnostico_extendido','Diagnostico_clasificacion']
df = df.merge(df_diag[cols], how='left', on='Diagnostico_transcrito')
df[['ID'] +cols]

,ID,Diagnostico_transcrito,Diagnostico_estandar,Diagnostico_extendido,Diagnostico_clasificacion
0,BO_1968_00001,S-D,S-D,sin datos,NaN
1,BO_1968_00002,S-D,S-D,sin datos,NaN
2,BO_1968_00003,S-D,S-D,sin datos,NaN
3,BO_1968_00004,S-D,S-D,sin datos,NaN
4,BO_1968_00005,S-D,S-D,sin datos,NaN
...,...,...,...,...,...
96839,BO_1982_07489,S-D,S-D,sin datos,NaN
96840,BO_1982_07490,S-D,S-D,sin datos,NaN
96841,BO_1982_07491,S-D,S-D,sin datos,NaN
96842,BO_1982_07492,S-D,S-D,sin datos,NaN


In [28]:
cols = ['Diagnostico_estandar','Diagnostico_extendido']
df[cols] = df[cols].replace({'S-D':nan, 'sin datos':nan})

for col in ['Diagnostico_estandar','Diagnostico_clasificacion']:
    df[col] = df[col].astype("category")

df[cols].value_counts(dropna=False).head(10)

Diagnostico_estandar  Diagnostico_extendido           
NaN                   NaN                                 55901
TM                    traumatismo multiple                 7525
TCE                   traumatismo craneo encefalico        5992
BN                    bronconeumonia                       1856
CVG                   congestion visceral generalizada     1329
TCT                   traumatismo craneo torax             1185
HPAF                  herida arma fuego                    1001
HPAFC                 herida arma fuego craneo              944
Dispensa              dispensa                              921
Quem                  quemaduras                            816
Name: count, dtype: int64

## Ordenar y guardar datos

Antes de guardar los datos veamos las columnas, esto nos dará una idea de que hemos hecho.

In [29]:
df.dtypes

ID                                      object
Numero_progresivo_transcrito            object
Nombre_completo_transcrito              object
Primer_apellido                         object
Segundo_apellido                        object
Nombres_propios                         object
Fecha_transcrito                        object
Expediente_SEMEFO_transcrito            object
Procedencia_transcrito                  object
Numero_acta_transcrito                  object
Diagnostico_transcrito                  object
Sexo                                  category
Edad_transcrito                         object
Tipo_restos                           category
Bitacora_ingresos                     category
Foja_transcrito                         object
Observaciones                           object
Identificacion                        category
Fecha_estandar                  datetime64[ns]
Fecha_año                             category
Fecha_mes                             category
Fecha_semana 

Al agregar columnas estas se ponen al final, por lo que sería bueno ordenarlas. Además, hay columnas de trabajo, cómo 'Edad_str' y 'Edad_int' que no son necesarias.

In [30]:
df = df[['ID', 'Numero_progresivo_transcrito', 
         'Nombre_completo_transcrito', 'Primer_apellido', 'Segundo_apellido', 'Nombres_propios',
         'Fecha_transcrito', 'Fecha_estandar', 'Fecha_año', 'Fecha_mes', 'Fecha_semana', 'Fecha_diasemana', 
         'Sexo', 'Edad_transcrito', 'Edad_estandar', 'Edad_grupo', 
         'Tipo_restos', 'Identificacion', 
         'Diagnostico_transcrito', 'Diagnostico_estandar', 'Diagnostico_extendido', 'Diagnostico_clasificacion',
         'Procedencia_transcrito', 'Procedencia_estandar', 'Procedencia_extendido', 'Procedencia_alcaldia', 'Procedencia_clasificacion',
         'Numero_acta_transcrito', 'Expediente_SEMEFO_transcrito', 
         'Bitacora_ingresos', 'Foja_transcrito', 'Observaciones',
  ]]
df.tail()

,ID,Numero_progresivo_transcrito,Nombre_completo_transcrito,Primer_apellido,Segundo_apellido,Nombres_propios,Fecha_transcrito,Fecha_estandar,Fecha_año,Fecha_mes,...,Procedencia_transcrito,Procedencia_estandar,Procedencia_extendido,Procedencia_alcaldia,Procedencia_clasificacion,Numero_acta_transcrito,Expediente_SEMEFO_transcrito,Bitacora_ingresos,Foja_transcrito,Observaciones
96839,BO_1982_07489,S-D,placenta,NaN,NaN,NaN,1982-06-05,1982-06-05,1982.0,6.0,...,15a,GAM-2,Coord Territorial 2 Gustavo A Madero (Col Arag...,Gustavo A Madero,Judicial,960,3079,semefo_df_bo_1982,156,NaN
96840,BO_1982_07490,S-D,5 dedos del pie derecho de desconocido,NaN,NaN,NaN,1982-06-05,1982-06-05,1982.0,6.0,...,32a,COY-2,Coord Territorial 2 Coyoacan (Col Romero de Te...,Coyoacan,Judicial,950,3060,semefo_df_bo_1982,156,NaN
96841,BO_1982_07491,S-D,dedo de desconocido,NaN,NaN,NaN,1982-11-19,1982-11-19,1982.0,11.0,...,32a,COY-2,Coord Territorial 2 Coyoacan (Col Romero de Te...,Coyoacan,Judicial,2005,6389,semefo_df_bo_1982,156,NaN
96842,BO_1982_07492,S-D,4 dedos de desconocido,NaN,NaN,NaN,1982-11-28,1982-11-28,1982.0,11.0,...,27a,XO-2,Coord Territorial 2 Xochimilco (Barrio de San ...,Xochimilco,Judicial,959,6528,semefo_df_bo_1982,156,NaN
96843,BO_1982_07493,S-D,osamenta de desconocido,NaN,NaN,NaN,1982-10-11,1982-10-11,1982.0,10.0,...,9a,MH-1,Coord Territorial 1 Miguel Hidalgo – Hospital ...,Miguel Hidalgo,Hospital,4193,5629,semefo_df_bo_1982,156,no se recibio necropsia. es una osamenta.


Hagamos un nuevo profile para ver el comportamiento de los datos limpios

In [31]:
from ydata_profiling import ProfileReport

file_profile = "profiles/HBO_profile_clean.html"
prof = ProfileReport( df, correlations=None, interactions=None ) 
prof.to_file(output_file=file_profile)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|███████████████████████████████████████████| 32/32 [00:03<00:00,  8.86it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]


Nosotros guardaremos los datos limpios en la carpeta `data_clean` como un CSV un `pickle`.

In [32]:
from joblib import dump

df = df.set_index('ID')

file_out_pickle = "data_clean/HBO_clean.pkl"
with open(file_out_pickle, 'wb') as f:
    dump(df, f)
    
file_out = "data_clean/HBO_clean.csv"
df.to_csv(file_out)

**Done**